In [1]:
from konlpy.tag import Okt
from konlpy.tag import Kkma
from kiwipiepy import Kiwi
import pandas as pd
import numpy as np
import koreanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from PIL import Image 
import re
from collections import Counter
from emoji import core

In [20]:
df = pd.read_csv(
    "../data/saramin_merge_sections_split_v1.csv"
)

#df = df.drop(columns=["Unnamed: 0"])

df.head()

,keyword,title,company,location,career,education,work_type,deadline,link,rec_idx,keyword_text,crawled_at,corp_domain,totaltext,responsibilities,qualifications,preferred_quali,hiring_process,benefits,salary
0,AI 에이전트,[AI기반 특허 데이터분석 플랫폼 기업] 프론트엔드 개발자,에이피써치,서울 강남구,경력 7~15년,대졸↑,정규직,채용시,https://www.saramin.co.kr/zf_user/jobs/relay/v...,52982351,프론트엔드 헤드헌팅,2026-02-02 12:27:26,23명 (2017년 기준),[AI기반 특허 데이터분석 플랫폼 기업] 프론트엔드 개발자\n프론트엔드 개발자 (1...,- 사용자 경험을 극대화하기 위한 창의적이고 실용적인 기능 설계 및 구현\n- 원하...,- Frontend 개발 경험을 7년 이상 보유하고 있으신 분\n- JavaScri...,- TypeScript에 대한 경험과 이해가 깊은 분\n- 대규모 프로젝트나 복잡한...,- 국문 이력서 및 경력기술서 (Word로 하나의 파일로 작성)\n- 이력서 제출시...,- 고용형태 : 정규직,추후협의\n면접 후 결정
1,AI 에이전트,[자율주행] AI Engineer / 3D Auto-labeling,(주)비저너리,서울 강남구,경력무관,석사↑,정규직,채용시,https://www.saramin.co.kr/zf_user/jobs/relay/v...,52981267,AI(인공지능) AI(인공지능),2026-02-02 12:20:37,응용 소프트웨어 개발 및 공급업,[자율주행] AI Engineer | 3D Auto-labeling / Spatia...,합류하면 풀게 될 문제\n비저너리의 데이터 자동 가공 파이프라인은 이미 동작하고 있...,필수\n관련 전공 석사 이상 학력 혹은 그에 준하는 경력 및 실력\n딥러닝 프로젝트...,Autolabeling / weak supervision / active learn...,지원서 접수 → (선택) 커피챗 → 기술 인터뷰 → 필요 시 추가 인터뷰(컬처/오프...,"Daily scrum으로 개발 우선순위 결정, 서로 피드백\n코드 리뷰를 통해 더 ...",스톡옵션 제공
2,AI 에이전트,[로봇 솔루션기업] 모바일로봇 교통 관제 시스템 설계 및 개발자,(주)맨토스파워,서울 마포구,경력2년↑,대졸↑,정규직,~ 02/17(화),https://www.saramin.co.kr/zf_user/jobs/relay/v...,52980063,AI(인공지능) AI(인공지능) 로봇설계 알고리즘 MAPF,2026-02-02 12:11:50,임시 및 일용 인력 공급업,[코스닥상장/로봇 소프트웨어 개발사] \n모바일로봇 교통 관제 시스템 설계 및 개발...,"- 멀티 로봇 경로 계획(Multi-Agent Path Finding, MAPF) ...","- 해당 분야 석사(신입) 또는 관련 분야 경력 2년 이상\n- 성실하고, 다양한 ...",📋 우대사항,2026-02-02 10시 ~ 2026-02-17 24시\n사람인 온라인 이력서\n...,모바일로봇 교통 관제 시스템 설계 및 개발 경력직 채용\n모바일로봇 교통 관제 시스...,NaN
3,AI 에이전트,Microsoft 365 Copilot 및 Agent 개발 컨설턴트,(주)무늬랩스,서울 송파구,경력2년↑,초대졸↑,정규직,~ 03/04(수),https://www.saramin.co.kr/zf_user/jobs/relay/v...,52979929,기술지원 백엔드/서버개발 AI(인공지능) 데이터분석가 데이터엔지니어,2026-02-02 12:22:23,경영 컨설팅업,Microsoft 365 Copilot 및 Agent 개발 컨설턴트\nM365 Co...,- 고객 맞춤형 M365 Copilot & Agent 활용 방안 제안 및 Copil...,"- 경력 2년 이상\n- 대학졸업(2,3년)이상\n- Copilot Agent 기획...","- Copilot Studio (Builder), Power Automate, Po...",서류전형 → 1차면접 (부문 리더) → 2차면접 (대표이사) → 최종합격\n- 이력...,"- 고용형태 : 정규직\n주 5일, 오전 9시 - 6시\n“일과 삶의 균형과 조화를...",협의 후 결정
4,AI 에이전트,35. AI Platform Engineer [대기업 게임사],베스트네트워크㈜,서울 강남구,경력2년↑,대졸↑,정규직,~ 02/17(화),https://www.saramin.co.kr/zf_user/jobs/relay/v...,52979323,AI(인공지능) 기술연구,2026-02-02 12:22:02,경영 컨설팅업,35. AI Platform Engineer [대기업 게임사]\nAI Platfor...,- 사내 유저들의 워크플로우를 분석· 개선하여 공통 문제를 해결하고 획기적인 생산성...,- 3년 이상 경력\n- 소프트웨어 엔지니어링: FE → BE → 인프라 배포/모니...,"- LangChain, OpenAI API 등 LLM 활용 경험\n- Tool Ca...",사람인 온라인 이력서\n사람인 입사지원\n서류전형 → 1차면접 → 2차면접 → 최종합격,- 고용형태 : 정규직,면접 후 결정


In [21]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   keyword           478 non-null    str  
 1   title             478 non-null    str  
 2   company           478 non-null    str  
 3   location          478 non-null    str  
 4   career            478 non-null    str  
 5   education         478 non-null    str  
 6   work_type         478 non-null    str  
 7   deadline          478 non-null    str  
 8   link              478 non-null    str  
 9   rec_idx           478 non-null    int64
 10  keyword_text      478 non-null    str  
 11  crawled_at        478 non-null    str  
 12  corp_domain       441 non-null    str  
 13  totaltext         478 non-null    str  
 14  responsibilities  478 non-null    str  
 15  qualifications    387 non-null    str  
 16  preferred_quali   328 non-null    str  
 17  hiring_process    386 non-null    str  
 18  b

In [22]:
#주요업무 토크나이징

kiwi = Kiwi()
index = 0
df["respons_text"] = None
for sent in df["responsibilities"]:
    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)
    result = kiwi.tokenize(clean_sent)
    sub_result = []    
    for x in result:
        word = x.form
        pos = x.tag
        if pos[0] in ["J", "E", "X"] or len(word) <= 1:
            continue
        sub_result.append(word)
    print(f"index: {index}, sub_list: {sub_result}")
    if len(sub_result) > 0:
        df.at[index, "respons_text"] = sub_result

    index += 1


df["respons_text"]


index: 0, sub_list: ['사용자', '경험', '극대', '위하', '창의', '실용', '기능', '설계', '구현', '결과', '도달', '자연어', '검색', '시각', '실시간', '반응', '서비스', 'AI', '에이전트', '통하', '자동', '분류', '문서', '생성', '설계', '구현', '신규', '서비스', '기획', '개발', '최신', '기술', '적용', '코드', '개선', 'React', 'Nextjs', '기반', '컴포넌트', '개발']
index: 1, sub_list: ['합류', '문제', '비저너리', '데이터', '자동', '가공', '파이프라인', '이미', '동작', '글로벌', '벤치마크', '기준', 'SOTA', '수준', '자체', '모델', '보유', '지금', '필요', '만들', '작업', '여러', 'Perception', '태스크', 'edge', 'case', '자동', '파악', '개선', '루프', '구축', '핵심', '합류', '적어도', '하나', '태스크', '처음', '책임지', '고객', '제공', '가능', '수준', '고도', '우리', '제품', '우선순위', '고객', '문제', '미리', '빠르', '가치', '만들', '미션', '이미', '동작', '자동', '가공', '파이프라인', 'Enterprise', '수준', '정확도', '일관', '일반', '재현', '개선', '고객', '도메인', '센서', '변화', '동작', '평가', '체계', '실패', '케이스', '정의', '개선', '루프', '구축', '새롭', '태스크', '센서', '환경', '확장', '구간', '문제', '한계', '조건', '뾰족', '정의', '자원', '제약', '사항', '고려', '빠르', '프로', '타이핑', '운영', '가능', '수준', '만들', 'Perception', '태스크', '자동', '가공', '모델', '개발', '학습', '파이프라인

0      [사용자, 경험, 극대, 위하, 창의, 실용, 기능, 설계, 구현, 결과, 도달, ...
1      [합류, 문제, 비저너리, 데이터, 자동, 가공, 파이프라인, 이미, 동작, 글로벌...
2      [멀티, 로봇, 경로, 계획, MultiAgent, Path, Finding, MA...
3      [고객, 맞춤, Copilot, Agent, 활용, 방안, 제안, Copilot, ...
4      [사내, 유저, 워크, 플로우, 분석, 개선, 공통, 문제, 해결, 획기적, 생산,...
                             ...                        
473    [지원금, 사용, 내역, 정리, 증빙, 자료, 준비, 결과, 보고서, 문서, 작성,...
474    [코딧, 프로, 고객, 정책, 규제, 니즈, 해결, 위하, 플랫폼, 개발, 운영, ...
475    [변하, 상황, 맞추, 다양, 업무, 경험, 업무, 효율, 높이, 환경, 제공, 웰...
476    [코딧, 프로, 고객, 정책, 규제, 니즈, 해결, 위하, 플랫폼, 개발, 운영, ...
477    [코딧이, 운영, 주요, 서비스, 대하, FrontendBackend, QA, 개발...
Name: respons_text, Length: 478, dtype: object

In [23]:
#자격/필수 토크나이징

kiwi = Kiwi()
index = 0
df["qualifi_text"] = None
for sent in df["qualifications"]:
    sent = "" if pd.isna(sent) else str(sent)
    clean_sent = re.sub("[^a-zA-Z가-힣0-9\s]", "", sent)
    result = kiwi.tokenize(clean_sent)
    sub_result = []    
    for x in result:
        word = x.form
        pos = x.tag
        if pos[0] in ["J", "E", "X"] or len(word) <= 1:
            continue
        sub_result.append(word)
    print(f"index: {index}, sub_list: {sub_result}")
    if len(sub_result) > 0:
        df.at[index, "qualifi_text"] = sub_result

    index += 1


df["qualifi_text"]

index: 0, sub_list: ['Frontend', '개발', '경험', '이상', '보유', 'JavaScript', 'React', '대하', '이해', '갖추', '계시', 'Nextjs', '기반', '프로젝트', '경험', '서비스', '대하', '이해', '보유', '문제', '해결', '성능', '최적', '능력', '보이', '새롭', '기술', '빠르', '적응', '활용', '능력', '보유', '계시', '동료', '의견', '존중', '논리', '소통', '통하', '합리', '기술', '의사', '결정', '함께', '이끌', '경력', '이상', '15', '이하', '학교', '졸업', '이상']
index: 1, sub_list: ['필수', '관련', '전공', '석사', '이상', '학력', '혹은', '준하', '경력', '실력', '러닝', '프로젝트', '학습', '평가', '개선', '결과물', '주도', '경험', 'vision', '또는', '멀티', '센서', '데이터', '다루', '경험', '연구', '산업', '무관', '논문', '오픈', '소스', '아이디어', '빠르', '구현', '실험', '의사', '결정', '연결', '경험', '이런', '주도', '문제', '정의', '해결', '시키', '아니', '문제', '스스로', '파악', '방향', '제시', '모르', '영역', '탐구', '이런', '수동', '지시', '기다리', '정하', '범위']
index: 2, sub_list: ['해당', '분야', '석사', '신입', '또는', '관련', '분야', '경력', '이상', '성실', '다양', '이해관계자', '원활', '협업', '커뮤니케이션', '가능', '알고리즘', '설계', '최적', '대하', '이해', '데이터', '구조', '경로', '탐색', '알고리즘', '대하', '이해', 'JAVA', '또는', '유사', '객체', '지향', '언어', '개발', '역량',

0      [Frontend, 개발, 경험, 이상, 보유, JavaScript, React, ...
1      [필수, 관련, 전공, 석사, 이상, 학력, 혹은, 준하, 경력, 실력, 러닝, 프...
2      [해당, 분야, 석사, 신입, 또는, 관련, 분야, 경력, 이상, 성실, 다양, 이...
3      [경력, 이상, 대학, 졸업, 23, 이상, Copilot, Agent, 기획, 개...
4      [이상, 경력, 소프트웨어, 엔지니어링, FE, BE, 인프라, 배포, 모니터링, ...
                             ...                        
473    [정부, 기관, 요구, 항목, 맞추, 자료, 준비, 선정, 사업, 운영, 위하, 코...
474    [필수, 이력서, 자기소개, 성장, 배경, 경험, 업무, 대하, 작성, 필수, 포트...
475    [필수, 이력서, 자기소개, 성장, 배경, 경험, 업무, 위주, 작문, 역량, 충분...
476    [이상, 콘텐츠, 마케팅, 경력, 보유, 필수, 이력서, 자기소개, 성장, 배경, ...
477                                                 None
Name: qualifi_text, Length: 478, dtype: object

In [24]:
#우대사항 토크나이징

kiwi = Kiwi()
index = 0
df["preferred_text"] = None
for sent in df["preferred_quali"]:
    sent = "" if pd.isna(sent) else str(sent)
    clean_sent = re.sub("[^a-zA-Z가-힣0-9\s]", "", sent)
    result = kiwi.tokenize(clean_sent)
    sub_result = []    
    for x in result:
        word = x.form
        pos = x.tag
        if pos[0] in ["J", "E", "X"] or len(word) <= 1:
            continue
        sub_result.append(word)
    print(f"index: {index}, sub_list: {sub_result}")
    if len(sub_result) > 0:
        df.at[index, "preferred_text"] = sub_result

    index += 1


df["preferred_text"]

index: 0, sub_list: ['TypeScript', '대하', '경험', '이해', '규모', '프로젝트', '업무', '성공', '수행', '경험', '분석', '설계', '주도', '진행', '경험', '다양', '프론트', '엔드', '프레임', '워크', '활용', '경험', '디자인', '시스템', '혹은', '컴포넌트', '라이브러리', '구축', '경험', '수준', '도메인', '지식', '필요', '서비스', '구축', '경험']
index: 1, sub_list: ['Autolabeling', 'weak', 'supervision', 'active', 'learning', 'uncertainty', '관련', '경험', '규모', '데이터', '처리', '실험', '추적', '버저', '경험', '자율', '주행', '도메인', '지식']
index: 2, sub_list: ['우대', '사항']
index: 3, sub_list: ['Copilot', 'Studio', 'Builder', 'Power', 'Automate', 'Power', 'BI', '전문가', 'Copilot', 'Agent', '개발', '프로젝트', 'PM', '경험', '보유자', 'SI', '회사', '또는', '365', '구축', 'AI', 'Agent', '개발자', '컨설턴트', '경력', '보유자', '기업', '사내', '365', 'Copilot', '구축', '운영', '담당자', '경력', '보유자', 'Microsoft', '관련', '전문', '자격증', '보유자']
index: 4, sub_list: ['LangChain', 'OpenAI', 'API', 'LLM', '활용', '경험', 'Tool', 'Calling', 'Agentic', 'Workflow', '설계', '개발', '경험', 'LangGraph', 'CrewAI', '최신', 'Agent', '프레임', '워크', '사용', '경험', '사내', '시스템', '통

0      [TypeScript, 대하, 경험, 이해, 규모, 프로젝트, 업무, 성공, 수행,...
1      [Autolabeling, weak, supervision, active, lear...
2                                               [우대, 사항]
3      [Copilot, Studio, Builder, Power, Automate, Po...
4      [LangChain, OpenAI, API, LLM, 활용, 경험, Tool, Ca...
                             ...                        
473                                                 None
474                                                 None
475                                                 None
476                                                 None
477                                                 None
Name: preferred_text, Length: 478, dtype: object

In [25]:
#채용절차/전형/지원방법/제출서류 등 토크나이징

kiwi = Kiwi()
index = 0
df["hiring_text"] = None
for sent in df["hiring_process"]:
    sent = "" if pd.isna(sent) else str(sent)
    clean_sent = re.sub("[^a-zA-Z가-힣0-9\s]", "", sent)
    result = kiwi.tokenize(clean_sent)
    sub_result = []    
    for x in result:
        word = x.form
        pos = x.tag
        if pos[0] in ["J", "E", "X"] or len(word) <= 1:
            continue
        sub_result.append(word)
    print(f"index: {index}, sub_list: {sub_result}")
    if len(sub_result) > 0:
        df.at[index, "hiring_text"] = sub_result

    index += 1


df["hiring_text"]

index: 0, sub_list: ['국문', '이력서', '경력', '기술', 'Word', '하나', '파일', '작성', '이력서', '제출', '이직', '사유', '퇴직', '사유', '최종', '연봉', '희망', '연봉', '필수', '기재', '요망', '기본급', '성과급', '각각', '기재', 'Email', 'wonmiahnapsearchcokr', '20260202', '11', '채용', '사람', '온라인', '이력서', '사람', '입사', '지원', '서류', '전형', '면접', '면접', '최종', '합격']
index: 1, sub_list: ['지원서', '접수', '선택', '커피챗', '기술', '인터뷰', '필요', '추가', '인터뷰', '컬처', '오프라인', '처우', '협의', '최종', '합류', '서류', '검토', '코딩', '테스트', '사전', '과제', '진행', '본인', '경험', '역량', '드러나', '자료', '제출', '이력서', '포트폴리오', '깃허브', '프로젝트', '설명', '형태', '자유', '문제', '어떻', '정의', '어떤', '선택', '결과', '무엇', '배우', '드러나', '자료', '선호', '본인', '경험', '문제', '얼마나', '깊이', '고민', '아직', '경험', '영역', '어떻', '사고', '접근', '가상', '시나리오', '바탕', '문제', '정의', '해결', '방향', '논의', '형태', '진행']
index: 2, sub_list: ['20260202', '10', '20260217', '24', '사람', '온라인', '이력서', '사람', '입사', '지원', '서류', '전형', '면접', '최종', '합격']
index: 3, sub_list: ['서류', '전형', '면접', '부문', '리더', '면접', '대표', '이사', '최종', '합격', '이력서', '필수', '출신', '학교', 'GPA', '포함', 

0      [국문, 이력서, 경력, 기술, Word, 하나, 파일, 작성, 이력서, 제출, 이...
1      [지원서, 접수, 선택, 커피챗, 기술, 인터뷰, 필요, 추가, 인터뷰, 컬처, 오...
2      [20260202, 10, 20260217, 24, 사람, 온라인, 이력서, 사람,...
3      [서류, 전형, 면접, 부문, 리더, 면접, 대표, 이사, 최종, 합격, 이력서, ...
4      [사람, 온라인, 이력서, 사람, 입사, 지원, 서류, 전형, 면접, 면접, 최종,...
                             ...                        
473    [httpswwwmkcokrnewssocietyview, 202105511674, ...
474    [httpswwwmkcokrnewssocietyview, 202105511674, ...
475    [httpswwwmkcokrnewssocietyview, 202105511674, ...
476    [httpswwwmkcokrnewssocietyview, 202105511674, ...
477    [httpswwwmkcokrnewssocietyview, 202105511674, ...
Name: hiring_text, Length: 478, dtype: object

In [26]:
#복지/근무환경/근무조건 등 토크나이징

kiwi = Kiwi()
index = 0
df["benefits_text"] = None
for sent in df["benefits"]:
    sent = "" if pd.isna(sent) else str(sent)
    clean_sent = re.sub("[^a-zA-Z가-힣0-9\s]", "", sent)
    result = kiwi.tokenize(clean_sent)
    sub_result = []    
    for x in result:
        word = x.form
        pos = x.tag
        if pos[0] in ["J", "E", "X"] or len(word) <= 1:
            continue
        sub_result.append(word)
    print(f"index: {index}, sub_list: {sub_result}")
    if len(sub_result) > 0:
        df.at[index, "benefits_text"] = sub_result

    index += 1


df["benefits_text"]

index: 0, sub_list: ['고용', '형태', '정규직']
index: 1, sub_list: ['Daily', 'scrum', '개발', '우선순위', '결정', '피드백', '코드', '리뷰', '통하', '구조', '논의', '기술', '의사', '결정', '자유', '토론', '서로', '판단', '존중', '상호', '존중', '호칭', '사용', '001600', 'Coding', 'Agent', '구독', '금액', '제한', 'RTX', 'PRO', '6000', 'GPU', '이상', 'GPU', '자원', '예산', '확보', '초기', '멤버', '제품', '아키텍처', '함께', '설계']
index: 2, sub_list: ['모바일', '로봇', '교통', '관제', '시스템', '설계', '개발', '경력', '채용', '모바일', '로봇', '교통', '관제', '시스템', '설계', '개발자']
index: 3, sub_list: ['고용', '형태', '정규직', '오전', '균형', '조화', '만들', '위하', '노력', '보험', '건강', '보험', '연금', '보험', '고용', '보험', '산재', '보험', '직원', '가족', '휴가비', '지원', '국내', '고급', '호텔', '리조트', '숙박', '식사', '제공', '직원', '자녀', '영유아', '학비', '어린이집', '보육료', '유치원', '유아', '학비', '지원', '결혼', '축하금', '출산', '축하금', '명절', '선물', '지급', '종합', '건강', '검진', '통신비', '지원', '모든', '팀원', '무늬랩스', '문화', '풍요', '만들', '기여', '성과', '중심', '프로페셔널', '리즈', '서로', '협력', '인간', '유대감', '균형', '중요', '생각']
index: 4, sub_list: ['고용', '형태', '정규직']
index: 5, sub_list: ['고용', '형태', 

0                                          [고용, 형태, 정규직]
1      [Daily, scrum, 개발, 우선순위, 결정, 피드백, 코드, 리뷰, 통하, ...
2      [모바일, 로봇, 교통, 관제, 시스템, 설계, 개발, 경력, 채용, 모바일, 로봇...
3      [고용, 형태, 정규직, 오전, 균형, 조화, 만들, 위하, 노력, 보험, 건강, ...
4                                          [고용, 형태, 정규직]
                             ...                        
473    [업무, 효율, 높이, 환경, 제공, 웰컴, 키트, 최신, 노트북, 듀얼, 모니터,...
474    [지원자, 이해, 위하, 아래, 서류, 제출, 필수, 이력서, 자기소개, 성장, 배...
475    [지원자, 이해, 위하, 아래, 서류, 제출, 필수, 이력서, 자기소개, 성장, 배...
476    [IT, 스타트업, 또는, AI, 관련, 산업, 이해도, 평소, 정책, 정치, 대하...
477    [코딧, CODIT, AI, 빅데이터, 기술, 의안, 법령, 정책, 정보, 분석, ...
Name: benefits_text, Length: 478, dtype: object

In [27]:
#급여/연봉/처우 등 토크나이징

kiwi = Kiwi()
index = 0
df["salary_text"] = None
for sent in df["salary"]:
    sent = "" if pd.isna(sent) else str(sent)
    clean_sent = re.sub("[^a-zA-Z가-힣0-9\s]", "", sent)
    result = kiwi.tokenize(clean_sent)
    sub_result = []    
    for x in result:
        word = x.form
        pos = x.tag
        if pos[0] in ["J", "E", "X"] or len(word) <= 1:
            continue
        sub_result.append(word)
    print(f"index: {index}, sub_list: {sub_result}")
    if len(sub_result) > 0:
        df.at[index, "salary_text"] = sub_result

    index += 1


df["salary_text"]

index: 0, sub_list: ['추후', '협의', '면접', '결정']
index: 1, sub_list: ['스톡옵션', '제공']
index: 2, sub_list: []
index: 3, sub_list: ['협의', '결정']
index: 4, sub_list: ['면접', '결정']
index: 5, sub_list: ['면접', '결정']
index: 6, sub_list: []
index: 7, sub_list: ['당사', '내규', '따르', '경력', '이전', '급여', '면접', '결정', '보험']
index: 8, sub_list: ['연봉', '3000']
index: 9, sub_list: []
index: 10, sub_list: []
index: 11, sub_list: ['이메일', '전송', 'SW', '개발', '리더', '성명', 'ooo', '기재', '요망', '면접', '합격', '경력', '사항', '감안', '최적', '연봉', '결정', '면접', '결정']
index: 12, sub_list: ['협의', '조정', '면접', '결정']
index: 13, sub_list: ['면접', '결정']
index: 14, sub_list: ['면접', '결정', '보험']
index: 15, sub_list: ['이메일', '전송', 'AI', '기술', '전략', '수립', '성명', 'ooo', '기재', '요망', '면접', '합격', '경력', '사항', '감안', '최적', '연봉', '결정', '면접', '결정']
index: 16, sub_list: ['연봉', '협의', '기존', '대비', '경력', '따르', '연봉', '가능']
index: 17, sub_list: ['면접', '결정']
index: 18, sub_list: ['회사', '굿즈', '선물', '드리', '가족', '함께', '즐기', '명절', '선물', '드리', '회식', '취향', '함께', '동료', '자유', 

0                                       [추후, 협의, 면접, 결정]
1                                             [스톡옵션, 제공]
2                                                   None
3                                               [협의, 결정]
4                                               [면접, 결정]
                             ...                        
473    [야근, 단위, 수당, 지급, 명절, 생일, 감사, 축하, 마음, 소정, 선물, 드...
474    [월세, 지원금, 제공, 회사, 인근, 거주자, 일정, 조건, 충족, 대상, 개별,...
475    [월세, 지원금, 제공, 회사, 인근, 거주자, 일정, 조건, 충족, 대상, 개별,...
476    [월세, 지원금, 제공, 회사, 인근, 거주자, 일정, 조건, 충족, 대상, 개별,...
477    [야근, 단위, 수당, 지급, 명절, 생일, 감사, 축하, 마음, 소정, 선물, 드...
Name: salary_text, Length: 478, dtype: object

In [28]:
df.head()

,keyword,title,company,location,career,education,work_type,deadline,link,rec_idx,...,preferred_quali,hiring_process,benefits,salary,respons_text,qualifi_text,preferred_text,hiring_text,benefits_text,salary_text
0,AI 에이전트,[AI기반 특허 데이터분석 플랫폼 기업] 프론트엔드 개발자,에이피써치,서울 강남구,경력 7~15년,대졸↑,정규직,채용시,https://www.saramin.co.kr/zf_user/jobs/relay/v...,52982351,...,- TypeScript에 대한 경험과 이해가 깊은 분\n- 대규모 프로젝트나 복잡한...,- 국문 이력서 및 경력기술서 (Word로 하나의 파일로 작성)\n- 이력서 제출시...,- 고용형태 : 정규직,추후협의\n면접 후 결정,"[사용자, 경험, 극대, 위하, 창의, 실용, 기능, 설계, 구현, 결과, 도달, ...","[Frontend, 개발, 경험, 이상, 보유, JavaScript, React, ...","[TypeScript, 대하, 경험, 이해, 규모, 프로젝트, 업무, 성공, 수행,...","[국문, 이력서, 경력, 기술, Word, 하나, 파일, 작성, 이력서, 제출, 이...","[고용, 형태, 정규직]","[추후, 협의, 면접, 결정]"
1,AI 에이전트,[자율주행] AI Engineer / 3D Auto-labeling,(주)비저너리,서울 강남구,경력무관,석사↑,정규직,채용시,https://www.saramin.co.kr/zf_user/jobs/relay/v...,52981267,...,Autolabeling / weak supervision / active learn...,지원서 접수 → (선택) 커피챗 → 기술 인터뷰 → 필요 시 추가 인터뷰(컬처/오프...,"Daily scrum으로 개발 우선순위 결정, 서로 피드백\n코드 리뷰를 통해 더 ...",스톡옵션 제공,"[합류, 문제, 비저너리, 데이터, 자동, 가공, 파이프라인, 이미, 동작, 글로벌...","[필수, 관련, 전공, 석사, 이상, 학력, 혹은, 준하, 경력, 실력, 러닝, 프...","[Autolabeling, weak, supervision, active, lear...","[지원서, 접수, 선택, 커피챗, 기술, 인터뷰, 필요, 추가, 인터뷰, 컬처, 오...","[Daily, scrum, 개발, 우선순위, 결정, 피드백, 코드, 리뷰, 통하, ...","[스톡옵션, 제공]"
2,AI 에이전트,[로봇 솔루션기업] 모바일로봇 교통 관제 시스템 설계 및 개발자,(주)맨토스파워,서울 마포구,경력2년↑,대졸↑,정규직,~ 02/17(화),https://www.saramin.co.kr/zf_user/jobs/relay/v...,52980063,...,📋 우대사항,2026-02-02 10시 ~ 2026-02-17 24시\n사람인 온라인 이력서\n...,모바일로봇 교통 관제 시스템 설계 및 개발 경력직 채용\n모바일로봇 교통 관제 시스...,NaN,"[멀티, 로봇, 경로, 계획, MultiAgent, Path, Finding, MA...","[해당, 분야, 석사, 신입, 또는, 관련, 분야, 경력, 이상, 성실, 다양, 이...","[우대, 사항]","[20260202, 10, 20260217, 24, 사람, 온라인, 이력서, 사람,...","[모바일, 로봇, 교통, 관제, 시스템, 설계, 개발, 경력, 채용, 모바일, 로봇...",None
3,AI 에이전트,Microsoft 365 Copilot 및 Agent 개발 컨설턴트,(주)무늬랩스,서울 송파구,경력2년↑,초대졸↑,정규직,~ 03/04(수),https://www.saramin.co.kr/zf_user/jobs/relay/v...,52979929,...,"- Copilot Studio (Builder), Power Automate, Po...",서류전형 → 1차면접 (부문 리더) → 2차면접 (대표이사) → 최종합격\n- 이력...,"- 고용형태 : 정규직\n주 5일, 오전 9시 - 6시\n“일과 삶의 균형과 조화를...",협의 후 결정,"[고객, 맞춤, Copilot, Agent, 활용, 방안, 제안, Copilot, ...","[경력, 이상, 대학, 졸업, 23, 이상, Copilot, Agent, 기획, 개...","[Copilot, Studio, Builder, Power, Automate, Po...","[서류, 전형, 면접, 부문, 리더, 면접, 대표, 이사, 최종, 합격, 이력서, ...","[고용, 형태, 정규직, 오전, 균형, 조화, 만들, 위하, 노력, 보험, 건강, ...","[협의, 결정]"
4,AI 에이전트,35. AI Platform Engineer [대기업 게임사],베스트네트워크㈜,서울 강남구,경력2년↑,대졸↑,정규직,~ 02/17(화),https://www.saramin.co.kr/zf_user/jobs/relay/v...,52979323,...,"- LangChain, OpenAI API 등 LLM 활용 경험\n- Tool Ca...",사람인 온라인 이력서\n사람인 입사지원\n서류전형 → 1차면접 → 2차면접 → 최종합격,- 고용형태 : 정규직,면접 후 결정,"[사내, 유저, 워크, 플로우, 분석, 개선, 공통, 문제, 해결, 획기적, 생산,...","[이상, 경력, 소프트웨어, 엔지니어링, FE, BE, 인프라, 배포, 모니터링, ...","[LangChain, OpenAI, API, LLM, 활용, 경험, Tool, Ca...","[사람, 온라인, 이력서, 사람, 입사, 지원, 서류, 전형, 면접, 면접, 최종,...","[고용, 형태, 정규직]","[면접, 결정]"


In [29]:
# AI 에이전트와 관련없는 값 삭제하기
del_data = [52923369 , 52895376, 52884267, 52884229, 52884099, 52833237, 52731483, 52731459, 52651246]

df = df[~df["rec_idx"].isin([52923369 , 52895376, 52884267, 52884229, 52884099, 52833237, 52731483, 52731459, 52651246])]





In [30]:
df.info()

<class 'pandas.DataFrame'>
Index: 469 entries, 0 to 477
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   keyword           469 non-null    str   
 1   title             469 non-null    str   
 2   company           469 non-null    str   
 3   location          469 non-null    str   
 4   career            469 non-null    str   
 5   education         469 non-null    str   
 6   work_type         469 non-null    str   
 7   deadline          469 non-null    str   
 8   link              469 non-null    str   
 9   rec_idx           469 non-null    int64 
 10  keyword_text      469 non-null    str   
 11  crawled_at        469 non-null    str   
 12  corp_domain       433 non-null    str   
 13  totaltext         469 non-null    str   
 14  responsibilities  469 non-null    str   
 15  qualifications    379 non-null    str   
 16  preferred_quali   320 non-null    str   
 17  hiring_process    378 non-null  

In [31]:
df.head()

df.to_csv(
    "../data/saramin_merge_sections_tokeni.csv", 
    index=False
)